In [2]:
# Load usefull libraries and helper functions
%run libs_and_utils

seed =  42
libraries loaded


In [3]:
# Load data
df_train = pd.read_csv(r'input/creditcard_train.csv')
df_train.sort_values(by=['Time'], inplace=True)
df_test = pd.read_csv(r'input/creditcard_test.csv')
df_test.sort_values(by=['Time'], inplace=True)

X_train = df_train.drop(columns='Class')
y_train = df_train.Class
X_test = df_test.drop(columns='Class')
y_test = df_test.Class
print('data_loaded')

data_loaded


# 5. Ensemble the Models
Simple averaging is used due to lack of time to do any better

In [18]:
models = [
    LGBMClassifier(class_weight='balanced', num_threads = -1, random_state=seed),
    ExtraTreesClassifier(class_weight='balanced', n_jobs = -1, random_state=seed),
    RandomForestClassifier(class_weight='balanced', n_jobs = -1, random_state=seed),
         ]

In [19]:
%%time
y_test_pred_probas=[]
for model in models:
    model_name = model.__class__.__name__
    # Load model and SMOTE best parameters
    model, alpha_over, k_neighbors = load_best_parameters_sampling(model)
    print('SMOTE_oversampling = ', alpha_over, '; SMOTE k_neighbors = ', k_neighbors)
    # Resample train using SMOTE best parameters
    over = SMOTE(random_state=42, sampling_strategy=alpha_over, k_neighbors=k_neighbors)
    X_train_res, y_train_res = over.fit_resample(X_train, y_train)
    #plot_auprc(model, X_train_res, y_train_res, X_test, y_test)
    # Refit the model on whole train data
    model.fit(X_train_res, y_train_res)
    y_test_pred_proba=model.predict_proba(X_test)[:,1]
    print(model_name, ' auprc = ', auprc(y_test, y_test_pred_proba))
    y_test_pred_probas.append(y_test_pred_proba)    

y_test_pred_proba_avg=np.mean(y_test_pred_probas)
print('Bagging_ensemble_auprc = ', auprc(y_test, y_test_pred_proba_avg))

LGBMClassifier optimized parameters: {'num_leaves': 618, 'reg_lambda': 0.2008734705666495, 'reg_alpha': 0.24515892466388628, 'max_depth': 86, 'min_data_in_leaf': 381, 'max_bin': 1076, 'learning_rate': 0.43704503206629275}
[LightGBM] [Warning] min_data_in_leaf is set=381, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=381
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
LGBMClassifier  auprc =  0.8358407450208815
ExtraTreesClassifier optimized parameters: {'n_estimators': 1461, 'max_depth': 30, 'criterion': 'gini', 'min_samples_split': 9}
ExtraTreesClassifier  auprc =  0.8276720393022088
RandomForestClassifier optimized parameters: {'n_estimators': 224, 'max_depth': 30, 'criterion': 'entropy', 'min_samples_split': 8, 'max_features': 30}
RandomForestClassifier  auprc =  0.8083044993405659


TypeError: Singleton array 0.004085866365012689 cannot be considered a valid collection.